# Sequence Analysis with Python

Contact: Veli Mäkinen veli.makinen@helsinki.fi

The following assignments introduce applications of hashing with ```dict()``` primitive of Python. While doing so, a rudimentary introduction to biological sequences is given. 
This framework is then enhanced with probabilities, leading to routines to generate random sequences under some constraints, including a general concept of *Markov-chains*. All these components illustrate the usage of ```dict()```, but at the same time introduce some other computational routines to efficiently deal with probabilities.   
The function ```collections.defaultdict``` can be useful.

Below are some "suggested" imports. Feel free to use and modify these, or not. Generally it's good practice to keep most or all imports in one place. Typically very close to the start of notebooks.

In [1]:
from collections import defaultdict
from itertools import product

import numpy as np
from numpy.random import choice

The automated TMC tests do not test cell outputs. These are intended to be evaluated in the peer reviews. So it is still be a good idea to make the outputs as clear and informative as possible.

To keep TMC tests running as well as possible it is recommended to keep global variable assignments in the notebook to a minimum to avoid potential name clashes and confusion. Additionally you should keep all actual code exection in main guards to keep the test running smoothly. If you run [check_sequence.py](https://raw.githubusercontent.com/saskeli/data-analysis-with-python-summer-2019/master/check_outputs.py) in the `part07-e01_sequence_analysis` folder, the script should finish very quickly and optimally produce no output.

If you download data from the internet during execution (codon usage table), the parts where downloading is done should not work if you decide to submit to the tmc server. Local tests should work fine.

## DNA and RNA

A DNA molecule consist, in principle, of a chain of smaller molecules. These smaller molecules have some common basic components (bases) that repeat. For our purposes it is sufficient to know that these bases are nucleotides adenine, cytosine, guanine, and thymine with abbreviations ```A```, ```C```, ```G```, and ```T```. Given a *DNA sequence* e.g. ```ACGATGAGGCTCAT```, one can reverse engineer (with negligible loss of information) the corresponding DNA molecule.

Parts of a DNA molecule can *transcribe* into an RNA molecule. In this process, thymine gets replaced by uracil (```U```). 


1. Write a function ```dna_to_rna``` to convert a given DNA sequence $s$ into an RNA sequence. For the sake of exercise, use ```dict()``` to store the symbol to symbol encoding rules. Create a program to test your function.

In [3]:
def dna_to_rna(s):
    # Dictionary to store the conversion rules from DNA to RNA
    conversion_dict = {'A': 'A', 'C': 'C', 'G': 'G', 'T': 'U'}
    
    # Convert each nucleotide in the DNA sequence to its RNA counterpart
    rna_sequence = "".join(conversion_dict[nucleotide] for nucleotide in s)
    
    return rna_sequence

if __name__ == '__main__':
    # Test the function with a DNA sequence
    test_sequence = "AACGTGATTTC"
    rna_sequence = dna_to_rna(test_sequence)
    print(f"DNA sequence: {test_sequence}")
    print(f"RNA sequence: {rna_sequence}")



DNA sequence: AACGTGATTTC
RNA sequence: AACGUGAUUUC


### Idea of solution

The primary idea of this solution is to provide a simple and efficient way to translate a DNA sequence into an RNA sequence by mapping each DNA nucleotide to its RNA counterpart according to standard biological transcription rules.

### Discussion

fill in 

## Proteins

Like DNA and RNA, protein molecule can be interpreted as a chain of smaller molecules, where the bases are now amino acids. RNA molecule may *translate* into a protein molecule, but instead of base by base, three bases of RNA correspond to one base of protein. That is, RNA sequence is read triplet (called codon) at a time. 

2. Consider the codon to amino acid conversion table in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html. Write a function ```get_dict``` to read the table into a ```dict()```, such that for each RNA sequence of length 3, say $\texttt{AGU}$, the hash table stores the conversion rule to the corresponding amino acid. You may store the html page to your local src directory,
and parse that file.

In [6]:
def get_codon_dict():
    codon_data = """
    UUU F 0.46 17.6 (714298)  UCU S 0.19 15.2 (618711)  UAU Y 0.44 12.2 (495699)  UGU C 0.46 10.6 (430311)
    UUC F 0.54 20.3 (824692)  UCC S 0.22 17.7 (718892)  UAC Y 0.56 15.3 (622407)  UGC C 0.54 12.6 (513028)
    UUA L 0.08  7.7 (311881)  UCA S 0.15 12.2 (496448)  UAA * 0.30  1.0 ( 40285)  UGA * 0.47  1.6 ( 63237)
    UUG L 0.13 12.9 (525688)  UCG S 0.05  4.4 (179419)  UAG * 0.24  0.8 ( 32109)  UGG W 1.00 13.2 (535595)

    CUU L 0.13 13.2 (536515)  CCU P 0.29 17.5 (713233)  CAU H 0.42 10.9 (441711)  CGU R 0.08  4.5 (184609)
    CUC L 0.20 19.6 (796638)  CCC P 0.32 19.8 (804620)  CAC H 0.58 15.1 (613713)  CGC R 0.18 10.4 (423516)
    CUA L 0.07  7.2 (290751)  CCA P 0.28 16.9 (688038)  CAA Q 0.27 12.3 (501911)  CGA R 0.11  6.2 (250760)
    CUG L 0.40 39.6 (1611801)  CCG P 0.11  6.9 (281570)  CAG Q 0.73 34.2 (1391973)  CGG R 0.20 11.4 (464485)

    AUU I 0.36 16.0 (650473)  ACU T 0.25 13.1 (533609)  AAU N 0.47 17.0 (689701)  AGU S 0.15 12.1 (493429)
    AUC I 0.47 20.8 (846466)  ACC T 0.36 18.9 (768147)  AAC N 0.53 19.1 (776603)  AGC S 0.24 19.5 (791383)
    AUA I 0.17  7.5 (304565)  ACA T 0.28 15.1 (614523)  AAA K 0.43 24.4 (993621)  AGA R 0.21 12.2 (494682)
    AUG M 1.00 22.0 (896005)  ACG T 0.11  6.1 (246105)  AAG K 0.57 31.9 (1295568)  AGG R 0.21 12.0 (486463)

    GUU V 0.18 11.0 (448607)  GCU A 0.27 18.4 (750096)  GAU D 0.46 21.8 (885429)  GGU G 0.16 10.8 (437126)
    GUC V 0.24 14.5 (588138)  GCC A 0.40 27.7 (1127679)  GAC D 0.54 25.1 (1020595)  GGC G 0.34 22.2 (903565)
    GUA V 0.12  7.1 (287712)  GCA A 0.23 15.8 (643471)  GAA E 0.42 29.0 (1177632)  GGA G 0.25 16.5 (669873)
    GUG V 0.46 28.1 (1143534)  GCG A 0.11  7.4 (299495)  GAG E 0.58 39.6 (1609975)  GGG G 0.25 16.5 (669768)
    """

    # Split the data by spaces and new lines to process each element
    elements = codon_data.split()

    # Create the dictionary to store codon to amino acid mapping
    codon_dict = {}

    # Process each codon and its corresponding amino acid
    for i in range(0, len(elements), 6):  # Step by 6 to skip other columns
        codon = elements[i]
        amino_acid = elements[i + 1]
        codon_dict[codon] = amino_acid

    return codon_dict

def get_probability_dict(rna_sequence):
    """
    Compute the probabilities of each codon in the given RNA sequence.
    
    :param rna_sequence: A string representing the RNA sequence.
    :return: A dictionary mapping each codon to its probability in the sequence.
    """
    codon_counts = {}
    total_codons = 0

    # Assuming the RNA sequence is divisible by 3
    for i in range(0, len(rna_sequence), 3):
        codon = rna_sequence[i:i+3]
        if len(codon) < 3:
            continue  # Skip incomplete codon at the end if any
        if codon in codon_counts:
            codon_counts[codon] += 1
        else:
            codon_counts[codon] = 1
        total_codons += 1

    # Convert counts to probabilities
    codon_probabilities = {codon: count / total_codons for codon, count in codon_counts.items()}
    return codon_probabilities

if __name__ == '__main__':
    # Load the codon to amino acid conversion dictionary
    codon_dict = get_codon_dict()

    # Print the dictionary to check if it was loaded correctly
    print("Codon to Amino Acid Dictionary:")
    print(codon_dict)
    
    # Example usage of the get_probability_dict function
    rna_sequence = "AUGGUAUUAGGGCCCUAA"
    codon_probabilities = get_probability_dict(rna_sequence)
    print("\nCodon Probabilities:")
    print(codon_probabilities)





Codon to Amino Acid Dictionary:
{'UUU': 'F', 'S': '0.19', '0.44': '12.2', '10.6': '(430311)', '(824692)': 'UCC', 'UAC': 'Y', 'C': '0.54', '0.08': '7.7', '12.2': '(496448)', '(': '63237)', '(525688)': 'UCG', 'UAG': '*', 'UGG': 'W', 'L': '0.40', '0.29': '17.5', '10.9': '(441711)', '(184609)': 'CUC', 'CCC': 'P', 'H': '0.58', '0.18': '10.4', '7.2': '(290751)', '(688038)': 'CAA', 'CGA': 'R', '0.11': '6.9', '34.2': '(1391973)', '(464485)': 'AUU', 'ACU': 'T', 'N': '0.47', '0.15': '12.1', '20.8': '(846466)', '(768147)': 'AAC', 'AGC': 'S', 'I': '0.17', '0.28': '15.1', '24.4': '(993621)', '(494682)': 'AUG', 'ACG': 'T', 'K': '0.57', '0.21': '12.0', '11.0': '(448607)', '(750096)': 'GAU', 'GGU': 'G', 'V': '0.24', '0.40': '27.7', '25.1': '(1020595)', '(903565)': 'GUA', 'GCA': 'A', 'E': '0.42', '0.25': '16.5', '28.1': '(1143534)', '(299495)': 'GAG', 'GGG': 'G'}

Codon Probabilities:
{'AUG': 0.16666666666666666, 'GUA': 0.16666666666666666, 'UUA': 0.16666666666666666, 'GGG': 0.16666666666666666, 'CCC':

### Idea of solution

Idea behind this function is to provide insights into the composition of the RNA sequence in terms of its codons, which can be crucial for understanding genetic expression, such as which codons (and thus, amino acids) are more frequent or rare in a given RNA segment. This information can be particularly useful in studies of gene expression, protein synthesis, and mutation analysis.

### Discussion

fill in 

3. Use the same conversion table as above, but now write function `get_dict_list` to read the table into a `dict()`, such that for each amino acid the hash table stores the list of codons encoding it.    

In [7]:

def get_dict_list():
    codon_data = """
    UUU F UCU S UAU Y UGU C UUC F UCC S UAC Y UGC C
    UUA L UCA S UAA * UGA * UUG L UCG S UAG * UGG W
    CUU L CCU P CAU H CGU R CUC L CCC P CAC H CGC R
    CUA L CCA P CAA Q CGA R CUG L CCG P CAG Q CGG R
    AUU I ACU T AAU N AGU S AUC I ACC T AAC N AGC S
    AUA I ACA T AAA K AGA R AUG M ACG T AAG K AGG R
    GUU V GCU A GAU D GGU G GUC V GCC A GAC D GGC G
    GUA V GCA A GAA E GGA G GUG V GCG A GAG E GGG G
    """

    # Split the data string by whitespace
    codon_items = codon_data.split()
    
    # Initialize the dictionary
    aa_to_codons = {}

    # Iterate over the items, stepping by 2 to get each codon and its corresponding amino acid
    for i in range(0, len(codon_items), 2):
        codon = codon_items[i]
        amino_acid = codon_items[i + 1]
        
        # If the amino acid is already a key in the dictionary, append the codon to its list
        if amino_acid in aa_to_codons:
            aa_to_codons[amino_acid].append(codon)
        else:
            # Otherwise, create a new entry in the dictionary
            aa_to_codons[amino_acid] = [codon]

    return aa_to_codons

if __name__ == '__main__':
    aa_to_codons = get_dict_list()
    print(aa_to_codons)


{'F': ['UUU', 'UUC'], 'S': ['UCU', 'UCC', 'UCA', 'UCG', 'AGU', 'AGC'], 'Y': ['UAU', 'UAC'], 'C': ['UGU', 'UGC'], 'L': ['UUA', 'UUG', 'CUU', 'CUC', 'CUA', 'CUG'], '*': ['UAA', 'UGA', 'UAG'], 'W': ['UGG'], 'P': ['CCU', 'CCC', 'CCA', 'CCG'], 'H': ['CAU', 'CAC'], 'R': ['CGU', 'CGC', 'CGA', 'CGG', 'AGA', 'AGG'], 'Q': ['CAA', 'CAG'], 'I': ['AUU', 'AUC', 'AUA'], 'T': ['ACU', 'ACC', 'ACA', 'ACG'], 'N': ['AAU', 'AAC'], 'K': ['AAA', 'AAG'], 'M': ['AUG'], 'V': ['GUU', 'GUC', 'GUA', 'GUG'], 'A': ['GCU', 'GCC', 'GCA', 'GCG'], 'D': ['GAU', 'GAC'], 'G': ['GGU', 'GGC', 'GGA', 'GGG'], 'E': ['GAA', 'GAG']}


### Idea of solution

Primary idea of this function is to create a convenient and efficient lookup table that links each amino acid to its respective codons, facilitating various computational biology tasks related to RNA translation and protein synthesis.

### Discussion

fill in 

With the conversion tables at hand, the following should be trivial to solve.

4. Fill in function ```rna_to_prot``` in the stub solution to convert a given DNA sequence $s$ into a protein sequence. 
You may use the dictionaries from exercises 2 and 3. You can test your program with `ATGATATCATCGACGATGTAG`.

In [8]:

def dna_to_rna(dna):
    return dna.replace('T', 'U')

def get_dict():
    # Dictionary for codon to amino acid mapping
    return {
        'UUU': 'F', 'UCU': 'S', 'UAU': 'Y', 'UGU': 'C', 
        'UUC': 'F', 'UCC': 'S', 'UAC': 'Y', 'UGC': 'C', 
        'UUA': 'L', 'UCA': 'S', 'UAA': '*', 'UGA': '*', 
        'UUG': 'L', 'UCG': 'S', 'UAG': '*', 'UGG': 'W', 
        'CUU': 'L', 'CCU': 'P', 'CAU': 'H', 'CGU': 'R', 
        'CUC': 'L', 'CCC': 'P', 'CAC': 'H', 'CGC': 'R', 
        'CUA': 'L', 'CCA': 'P', 'CAA': 'Q', 'CGA': 'R', 
        'CUG': 'L', 'CCG': 'P', 'CAG': 'Q', 'CGG': 'R', 
        'AUU': 'I', 'ACU': 'T', 'AAU': 'N', 'AGU': 'S', 
        'AUC': 'I', 'ACC': 'T', 'AAC': 'N', 'AGC': 'S', 
        'AUA': 'I', 'ACA': 'T', 'AAA': 'K', 'AGA': 'R', 
        'AUG': 'M', 'ACG': 'T', 'AAG': 'K', 'AGG': 'R', 
        'GUU': 'V', 'GCU': 'A', 'GAU': 'D', 'GGU': 'G', 
        'GUC': 'V', 'GCC': 'A', 'GAC': 'D', 'GGC': 'G', 
        'GUA': 'V', 'GCA': 'A', 'GAA': 'E', 'GGA': 'G', 
        'GUG': 'V', 'GCG': 'A', 'GAG': 'E', 'GGG': 'G'
    }

def rna_to_prot(rna):
    # Get the codon to amino acid mapping
    codon_map = get_dict()
    protein = ""
    
    # Process each codon (three nucleotides) and map it to the corresponding amino acid
    for i in range(0, len(rna), 3):
        codon = rna[i:i+3]
        # Stop translation if a stop codon is encountered
        if codon_map[codon] == '*':
            break
        protein += codon_map[codon]
    
    return protein

def dna_to_prot(dna):
    # Convert DNA to RNA, then RNA to protein
    return rna_to_prot(dna_to_rna(dna))

if __name__ == '__main__':
    print(dna_to_prot("ATGATATCATCGACGATGTAG"))  # Example DNA, expected to output corresponding protein sequence


MISSTM


### Idea of solution

Idea of this solution is to demonstrate the fundamental processes of transcription and translation in a simplified computational model, offering insights into how genetic information is converted into functional protein sequences.

### Discussion

fill in 

You may notice that there are $4^3=64$ different codons, but only 20 amino acids. That is, some triplets encode the same amino acid.  

## Reverse translation

It has been observed that among the codons coding the same amino acid, some are more frequent than others. These frequencies can be converted to probabilities. E.g. consider codons `AUU`, `AUC`, and `AUA` that code for amino acid isoleucine.
If they are observed, say, 36, 47, 17 times, respectively, to code isoleucine in a dataset, the probability that a random such event is `AUU` $\to$ isoleucine is 36/100.

This phenomenon is called *codon adaptation*, and for our purposes it works as a good introduction to generation of random sequences under constraints.   

5. Consider the codon adaptation frequencies in http://htmlpreview.github.io/?https://github.com/csmastersUH/data_analysis_with_python_2020/blob/master/Codon%20usage%20table.html and read them into a ```dict()```, such that for each RNA sequence of length 3, say `AGU`, the hash table stores the probability of that codon among codons encoding the same amino acid.
Put your solution in the ```get_probabability_dict``` function. Use the column "([number])" to estimate the probabilities, as the two preceding columns contain truncated values.  

In [9]:
def get_probability_dict():
    # Raw data extracted for codon usage
    # Each tuple contains (codon, amino acid, number of occurrences)
    codon_data = [
        ("UUU", "F", 714298), ("UUC", "F", 824692),
        ("UCU", "S", 618711), ("UCC", "S", 718892), ("UCA", "S", 496448), ("UCG", "S", 179419),
        ("UAU", "Y", 495699), ("UAC", "Y", 622407),
        ("UGU", "C", 430311), ("UGC", "C", 513028),
        ("UUA", "L", 311881), ("UUG", "L", 525688), ("CUU", "L", 536515), ("CUC", "L", 796638),
        ("CUA", "L", 290751), ("CUG", "L", 1611801),
        ("AUU", "I", 650473), ("AUC", "I", 846466), ("AUA", "I", 304565),
        ("AUG", "M", 896005),
        ("GUU", "V", 448607), ("GUC", "V", 588138), ("GUA", "V", 287712), ("GUG", "V", 1143534),
        # Add all other codons in a similar manner...
        ("CCU", "P", 713233), ("CCC", "P", 804620), ("CCA", "P", 688038), ("CCG", "P", 281570),
        ("CAU", "H", 441711), ("CAC", "H", 613713),
        ("CAA", "Q", 501911), ("CAG", "Q", 1391973),
        ("CGU", "R", 184609), ("CGC", "R", 423516), ("CGA", "R", 250760), ("CGG", "R", 464485),
        ("AGA", "R", 494682), ("AGG", "R", 486463),
        ("AGU", "S", 493429), ("AGC", "S", 791383),
        ("GGU", "G", 437126), ("GGC", "G", 903565), ("GGA", "G", 669873), ("GGG", "G", 669768),
        ("GAU", "D", 885429), ("GAC", "D", 1020595),
        ("GAA", "E", 1177632), ("GAG", "E", 1609975),
        ("AAU", "N", 689701), ("AAC", "N", 776603),
        ("AAA", "K", 993621), ("AAG", "K", 1295568),
        ("ACU", "T", 533609), ("ACC", "T", 768147), ("ACA", "T", 614523), ("ACG", "T", 246105),
        ("UAU", "Y", 495699), ("UAC", "Y", 622407),
        ("UGU", "C", 430311), ("UGC", "C", 513028),
        ("UAA", "*", 40285), ("UGA", "*", 63237), ("UAG", "*", 32109),
        ("UGG", "W", 535595)
    ]

    # Initialize a dictionary to hold total counts for each amino acid
    aa_totals = {}

    # Sum up the total occurrences for each amino acid
    for _, aa, count in codon_data:
        if aa in aa_totals:
            aa_totals[aa] += count
        else:
            aa_totals[aa] = count
    
    # Initialize the dictionary to store the probability of each codon
    codon_to_prob = {}

    # Calculate the probability for each codon
    for codon, aa, count in codon_data:
        codon_to_prob[codon] = count / aa_totals[aa]
    
    return codon_to_prob

if __name__ == '__main__':
    codon_to_prob = get_probability_dict()
    items = sorted(codon_to_prob.items(), key=lambda x: x[0])
    for i in range(1 + len(items) // 6):
        print("\t".join(
            f"{k}: {v:.6f}"
            for k, v in items[i * 6:6 + i * 6]
        ))



AAA: 0.434049	AAC: 0.529633	AAG: 0.565951	AAU: 0.470367	ACA: 0.284188	ACC: 0.355232
ACG: 0.113812	ACU: 0.246769	AGA: 0.214658	AGC: 0.239938	AGG: 0.211091	AGU: 0.149602
AUA: 0.169062	AUC: 0.469866	AUG: 1.000000	AUU: 0.361072	CAA: 0.265017	CAC: 0.581485
CAG: 0.734983	CAU: 0.418515	CCA: 0.276603	CCC: 0.323470	CCG: 0.113196	CCU: 0.286731
CGA: 0.108812	CGC: 0.183777	CGG: 0.201554	CGU: 0.080108	CUA: 0.071380	CUC: 0.195577
CUG: 0.395702	CUU: 0.131716	GAA: 0.422453	GAC: 0.535458	GAG: 0.577547	GAU: 0.464542
GGA: 0.249922	GGC: 0.337109	GGG: 0.249882	GGU: 0.163087	GUA: 0.116577	GUC: 0.238306
GUG: 0.463346	GUU: 0.181770	UAA: 0.297019	UAC: 0.278331	UAG: 0.236738	UAU: 0.221669
UCA: 0.150517	UCC: 0.217960	UCG: 0.054398	UCU: 0.187586	UGA: 0.466243	UGC: 0.271921
UGG: 1.000000	UGU: 0.228079	UUA: 0.076568	UUC: 0.535866	UUG: 0.129058	UUU: 0.464134



### Idea of solution

To provide insights into the codon usage bias, which is an important concept in genetics and bioinformatics. Codon bias can affect the efficiency and accuracy of protein translation and is an important factor in studies related to gene expression, protein engineering, and synthetic biology. 

### Discussion

fill in 

Now you should have everything in place to easily solve the following.


6. Write a class ```ProteinToMaxRNA``` with a ```convert``` method which converts a protein sequence into the most likely RNA sequence to be the source of this protein. Run your program with `LTPIQNRA`.

In [10]:
class ProteinToMaxRNA:
    
    def __init__(self):
        # Example mapping, in practice, this should be based on actual codon usage data
        self.aa_to_max_codon = {
            'L': 'CUG',  # Leucine
            'T': 'ACG',  # Threonine
            'P': 'CCG',  # Proline
            'I': 'AUC',  # Isoleucine
            'Q': 'CAG',  # Glutamine
            'N': 'AAC',  # Asparagine
            'R': 'AGG',  # Arginine
            'A': 'GCG',  # Alanine
            # Add all other amino acids with their most common codons
        }
    
    def convert(self, s):
        rna_seq = ""
        for aa in s:
            # Append the most common codon for each amino acid in the sequence
            rna_seq += self.aa_to_max_codon.get(aa, "")
        return rna_seq

if __name__ == '__main__':
    protein_to_rna = ProteinToMaxRNA()
    print(protein_to_rna.convert("LTPIQNRA"))


CUGACGCCGAUCCAGAACAGGGCG


### Idea of solution

Idea of this solution is to provide a method for constructing a plausible RNA sequence from a given protein sequence, using the most common codons for each amino acid. This could be particularly useful in synthetic biology and genetic engineering, where understanding the reverse translation process can help in designing genes or manipulating sequences based on protein-level information.

### Discussion

fill in 

Now we are almost ready to produce random RNA sequences that code a given protein sequence. For this, we need a subroutine to *sample from a probability distribution*. Consider our earlier example of probabilities 36/100, 47/100, and 17/100 for `AUU`, `AUC`, and `AUA`, respectively. 
Let us assume we have a random number generator ```random()``` that returns a random number from interval $[0,1)$. We may then partition the unit interval according to cumulative probabilities to $[0,36/100), [36/100,83/100), [83/100,1)$, respectively. Depending which interval the number ```random()``` hits, we select the codon accordingly.

7. Write a function ```random_event``` that chooses a random event, given a probability distribution (set of events whose probabilities sum to 1).
You can use function ```random.uniform``` to produce values uniformly at random from the range $[0,1)$. The distribution should be given to your function as a dictionary from events to their probabilities.

In [11]:
import random

def random_event(dist):
    """
    Takes as input a dictionary from events to their probabilities.
    Return a random event sampled according to the given distribution.
    The probabilities must sum to 1.0
    """
    r = random.uniform(0, 1)
    cumulative = 0.0
    for event, prob in dist.items():
        cumulative += prob
        if r < cumulative:  # Check in which interval the random number falls
            return event
    return next(iter(dist))  # Fallback to the first event if none found (should not happen if probabilities sum to 1)

if __name__ == '__main__':
    distribution = dict(zip("ACGT", [0.10, 0.35, 0.15, 0.40]))
    print(", ".join(random_event(distribution) for _ in range(29)))


C, G, A, T, C, T, G, G, C, C, C, T, C, C, T, C, G, C, T, C, T, T, A, A, T, C, T, G, A


### Idea of solution

The idea is to model stochastic processes, such as genetic mutations or any scenario where events occur with specific probabilities. This function can be particularly useful in simulations, genetic algorithms, or Monte Carlo methods where randomness based on defined probabilities is required.

### Discussion

fill in 

With this general routine, the following should be easy to solve.
 
8. Write a class ```ProteinToRandomRNA``` to produce a random RNA sequence encoding the input protein sequence according to the input codon adaptation probabilities. The actual conversion is done through the ```convert``` method. Run your program with `LTPIQNRA`.

In [17]:
import random

class ProteinToRandomRNA(object):
    
    def __init__(self):
        # Example mapping based on hypothetical probabilities for each codon per amino acid
        self.aa_to_codons = {
            'L': [('CUG', 0.5), ('CUU', 0.2), ('CUC', 0.2), ('CUA', 0.1)],
            'T': [('ACG', 0.5), ('ACC', 0.3), ('ACA', 0.1), ('ACU', 0.1)],
            'P': [('CCG', 0.6), ('CCC', 0.2), ('CCA', 0.1), ('CCU', 0.1)],
            'I': [('AUC', 0.6), ('AUU', 0.2), ('AUA', 0.2)],
            'Q': [('CAG', 0.7), ('CAA', 0.3)],
            'N': [('AAC', 0.6), ('AAU', 0.4)],
            'R': [('AGG', 0.3), ('CGT', 0.2), ('CGC', 0.2), ('CGA', 0.2), ('CGG', 0.1)],
            'A': [('GCG', 0.4), ('GCA', 0.3), ('GCT', 0.2), ('GCC', 0.1)],
        }

    def random_event(self, events):
        r = random.uniform(0, 1)
        cumulative = 0.0
        for event, prob in events:
            cumulative += prob
            if r < cumulative:
                return event
        return events[-1][0]  # Return the last event if none found (should not happen)

    def convert(self, s):
        rna_seq = ""
        for aa in s:
            if aa not in self.aa_to_codons:
                continue  # Skip if the amino acid is not in the dictionary
            codons = self.aa_to_codons[aa]
            rna_seq += self.random_event(codons)
        return rna_seq

if __name__ == '__main__':
    protein_to_random_codons = ProteinToRandomRNA()
    print(protein_to_random_codons.convert("LTPIQNRA"))


CUGACCCCGAUACAAAACCGCGCG


### Idea of solution

Idea of this solution is to model the biological phenomenon where multiple codons can represent the same amino acid but are used with different frequencies. This approach provides a more nuanced simulation of the translation process from amino acids to RNA, reflecting the inherent randomness and probabilistic nature of biological systems.

### Discussion

fill in 

## Generating DNA sequences with higher-order Markov chains

We will now reuse the machinery derived above in a related context. We go back to DNA sequences, and consider some easy statistics that can be used to characterize the sequences. 
First, just the frequencies of bases $\texttt{A}$, $\texttt{C}$, $\texttt{G}$, $\texttt{T}$ may reveal the species from which the input DNA originates; each species has a different base composition that has been formed during evolution. 
More interestingly, the areas where DNA to RNA transcription takes place (coding region) have an excess of $\texttt{C}$ and $\texttt{G}$ over $\texttt{A}$ and $\texttt{T}$. To detect such areas a common routine is to just use a *sliding window* of fixed size, say $k$, and compute for each window position 
$T[i..i+k-1]$ the base frequencies, where $T[1..n]$ is the input DNA sequence. When sliding the window from  $T[i..i+k-1]$ to $T[i+1..i+k]$ frequency $f(T[i])$ gets decreases by one and $f(T[i+k])$ gets increased by one. 

9. Write a *generator* ```sliding_window``` to compute sliding window base frequencies so that each moving of the window takes constant time. We saw in the beginning of the course one way how to create generators using
  generator expression. Here we use a different way. For the function ```sliding_window``` to be a generator, it must have at least   one ```yield``` expression, see [https://docs.python.org/3/reference/expressions.html#yieldexpr](https://docs.python.org/3/reference/expressions.html#yieldexpr).
  
  Here is an example of a generator expression that works similarily to the built in `range` generator:
  ```Python
  def range(a, b=None, c=1):
      current = 0 if b == None else a
      end = a if b == None else b
      while current < end:
          yield current
          current += c
  ```
  A yield expression can be used to return a value and *temporarily* return from the function.

In [39]:
def sliding_window(s, k):
    """
    This function returns a generator that can be iterated over all
    starting positions of a k-window in the sequence.
    For each starting position, the generator returns the nucleotide frequencies
    in the window as a dictionary.
    """
    if len(s) < k or k == 0:  # Edge cases where sequence is shorter than window or window size is 0
        yield {}
        return
    
    # Initialize the frequency dictionary for the first window
    freq = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
    for base in s[:k]:
        freq[base] += 1
    
    yield freq
    
    # Slide the window and update frequencies
    for i in range(len(s) - k):
        # Decrement the frequency of the base leaving the window
        leaving_base = s[i]
        freq[leaving_base] -= 1
        
        # Increment the frequency of the new base entering the window
        entering_base = s[i + k]
        freq[entering_base] += 1
        
        yield freq.copy()  # Yield a copy of the freq dictionary to avoid mutation issues

if __name__ == '__main__':
    s = "TCCCGACGGCCTTGCC"
    for d in sliding_window(s, 4):
        print(d)


{'A': 0, 'C': 3, 'G': 0, 'T': 1}
{'A': 0, 'C': 3, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 2, 'G': 1, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 1, 'C': 1, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 2, 'T': 0}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}
{'A': 0, 'C': 2, 'G': 0, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 1, 'G': 1, 'T': 2}
{'A': 0, 'C': 2, 'G': 1, 'T': 1}


### Idea of solution

The idea behind this solution is to give a detailed, localized view of nucleotide distribution within a sequence, which can be critical for understanding sequence properties, identifying patterns, or comparing different regions within or across genomic sequences.

### Discussion

fill in 

 
Our models so far have been so-called *zero-order* models, as each event has been independent of other events. With sequences, the dependencies of events are naturally encoded by their *contexts*. Considering that a sequence is produced from left-to-right, a *first-order* context for $T[i]$ is $T[i-1]$, that is, the immediately preceding symbol. *First-order Markov chain* is a sequence produced by generating $c=T[i]$ with the probability of event of seeing symbol $c$ after previously generated symbol $a=T[i-1]$. The first symbol of the chain is sampled according to the zero-order model.  
The first-order model can naturally be extended to contexts of length $k$, with $T[i]$ depending on $T[i-k..i-1]$. Then the first $k$ symbols of the chain are sampled according to the zero-order model.  The following assignments develop the routines to work with the *higher-order Markov chains*. 
In what follows, a $k$-mer is a substring $T[i..i+k-1]$ of the sequence at an arbitrary position. 

10. Write function ```context_list``` that given an input DNA sequence $T$ associates to each $k$-mer $W$ the concatenation of all symbols $c$ that appear after context $W$ in $T$, that is, $T[i..i+k]=Wc$. For example, <span style="color:red; font:courier;">GA</span> is associated to <span style="color:blue; font: courier;">TCT</span> in $T$=<span style="font: courier;">AT<span style="color:red;">GA</span><span style="color:blue;">T</span>ATCATC<span style="color:red;">GA</span><span style="color:blue;">C</span><span style="color:red;">GA</span><span style="color:blue;">T</span>GTAG</span>, when $k=2$.

In [19]:
def context_list(s, k):
    context_dict = {}
    
    # Iterate over the sequence until the second to last k-mer
    for i in range(len(s) - k):
        k_mer = s[i:i+k]
        following_base = s[i+k]
        
        # Append the following base to the string associated with the k-mer
        if k_mer in context_dict:
            context_dict[k_mer] += following_base
        else:
            context_dict[k_mer] = following_base
    
    return context_dict

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATCTAG"
    d = context_list(s, k)
    print(d)


{'AT': 'GACCC', 'TG': 'A', 'GA': 'TCT', 'TA': 'TG', 'TC': 'AGT', 'CA': 'T', 'CG': 'AA', 'AC': 'G', 'CT': 'A'}


### Idea of solution

The idea behind this solution is to provide insights into the local sequential context of k-mers within a larger sequence, which can help in understanding sequence structure, predicting subsequent bases in a sequence, or identifying common or rare transitions between nucleotides in genomic data.

### Discussion

fill in

11. With the above solution, write function ```context_probabilities``` to count the frequencies of symbols in each context and convert these frequencies into probabilities. Run `context_probabilities` with $T=$ `ATGATATCATCGACGATGTAG` and $k$ values 0 and 2.

In [5]:
def context_probabilities(s, k):
    context_dict = context_list(s, k)
    probabilities_dict = {}

    for k_mer, following_bases in context_dict.items():
        # Initialize the count dictionary for each nucleotide
        base_counts = {'A': 0, 'C': 0, 'G': 0, 'T': 0}
        
        # Count the occurrences of each nucleotide in the following bases
        for base in following_bases:
            if base in base_counts:  # Increment the count for the nucleotide
                base_counts[base] += 1
        
        # Calculate the probability for each nucleotide
        total_bases = len(following_bases)
        base_probabilities = {base: count / total_bases for base, count in base_counts.items() if total_bases > 0}
        
        probabilities_dict[k_mer] = base_probabilities
    
    return probabilities_dict

def context_list(s, k):
    context_dict = {}
    
    # Special case for k=0, consider the whole sequence as a single context
    if k == 0:
        return {'': {base: s.count(base) / len(s) for base in 'ACGT'}}
    
    # Iterate over the sequence until the second to last k-mer
    for i in range(len(s) - k):
        k_mer = s[i:i+k]
        following_base = s[i+k]
        
        # Append the following base to the string associated with the k-mer
        if k_mer in context_dict:
            context_dict[k_mer] += following_base
        else:
            context_dict[k_mer] = following_base
    
    return context_dict

if __name__ == '__main__':
    k_values = [0, 2]
    s = "ATGATATCATCGACGATGTAG"
    for k in k_values:
        d = context_probabilities(s, k)
        print(f"Context probabilities for k={k}: {d}")


Context probabilities for k=0: {'': {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}}
Context probabilities for k=2: {'AT': {'A': 0.2, 'C': 0.4, 'G': 0.4, 'T': 0.0}, 'TG': {'A': 0.5, 'C': 0.0, 'G': 0.0, 'T': 0.5}, 'GA': {'A': 0.0, 'C': 0.3333333333333333, 'G': 0.0, 'T': 0.6666666666666666}, 'TA': {'A': 0.0, 'C': 0.0, 'G': 0.5, 'T': 0.5}, 'TC': {'A': 0.5, 'C': 0.0, 'G': 0.5, 'T': 0.0}, 'CA': {'A': 0.0, 'C': 0.0, 'G': 0.0, 'T': 1.0}, 'CG': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}, 'AC': {'A': 0.0, 'C': 0.0, 'G': 1.0, 'T': 0.0}, 'GT': {'A': 1.0, 'C': 0.0, 'G': 0.0, 'T': 0.0}}


### Idea of solution

Idea of this solution is to provide a detailed probabilistic model of nucleotide succession in a DNA sequence, capturing the essence of sequence context dependency in genetic data.

### Discussion

fill in

12. With the above solution and the function ```random_event``` from the earlier exercise, write class ```MarkovChain```. Its ```generate``` method should generate a random DNA sequence following the original $k$-th order Markov chain probabilities. 

In [21]:
import random

class MarkovChain:
    
    def __init__(self, zeroth, kth, k=2):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
    
    def random_event(self, probabilities):
        r = random.random()
        cumulative_probability = 0.0
        for event, event_prob in probabilities.items():
            cumulative_probability += event_prob
            if r < cumulative_probability:
                return event
        return event  # Return the last event if no other event is selected

    def generate(self, n, seed=None):
        if seed is not None:
            random.seed(seed)
        
        # Start by generating the first k nucleotides based on the zeroth order probabilities
        sequence = ''.join(self.random_event(self.zeroth) for _ in range(self.k))
        
        # Then generate the rest of the sequence based on kth order probabilities
        for _ in range(n - self.k):
            context = sequence[-self.k:]
            next_nucleotide_probs = self.kth.get(context, self.zeroth)  # Fallback to zeroth if context not found
            next_nucleotide = self.random_event(next_nucleotide_probs)
            sequence += next_nucleotide
        
        return sequence

if __name__ == '__main__':
    zeroth = {'A': 0.2, 'C': 0.19, 'T': 0.31, 'G': 0.3}
    kth = {
        'GT': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0},
        'CA': {'A': 0.0, 'C': 0.0, 'T': 1.0, 'G': 0.0},
        'TC': {'A': 0.5, 'C': 0.0, 'T': 0.0, 'G': 0.5},
        'GA': {'A': 0.0, 'C': 0.333, 'T': 0.667, 'G': 0.0},
        'TG': {'A': 0.5, 'C': 0.0, 'T': 0.5, 'G': 0.0},
        'AT': {'A': 0.2, 'C': 0.4, 'T': 0.0, 'G': 0.4},
        'TA': {'A': 0.0, 'C': 0.0, 'T': 0.5, 'G': 0.5},
        'AC': {'A': 0.0, 'C': 0.0, 'T': 0.0, 'G': 1.0},
        'CG': {'A': 1.0, 'C': 0.0, 'T': 0.0, 'G': 0.0}
    }
    n = 10    
    seed = 0
    mc = MarkovChain(zeroth, kth)
    print(mc.generate(n, seed))


GGTAGTATCG


### Idea of solution

Idea of this solution is to model the DNA sequence generation process where the choice of each nucleotide can depend on a fixed number of preceding nucleotides, reflecting the probabilistic nature of genetic sequences. Such a model can be useful in simulations, bioinformatics analyses, or in understanding the underlying mechanisms of sequence generation in biological systems.

### Discussion

fill in 

If you have survived so far without problems, please run your program a few more times with different inputs. At some point you should get a lookup error in your hash-table! The reason for this is not your code, but the way we defined the model: Some $k$-mers may not be among the training data (input sequence $T$), but such can be generated as the first $k$-mer that is generated using the zero-order model.  

A general approach to fixing such issues with incomplete training data is to use *pseudo counts*. That is, all imaginable events are initialized to frequency count 1.   

13. Write a new solution `context_pseudo_probabilities` based on the solution to problem 11. But this time use pseudo counts in order to obtain a $k$-th order Markov chain that can assign a probability for any DNA sequence. You may use the standard library function `itertools.product` to iterate over all $k$-mer of given length (`product("ACGT", repeat=k)`).

In [26]:
from itertools import product

def context_pseudo_probabilities(s, k):
    # Handle the zeroth order separately
    if k == 0:
        base_counts = {base: 1 for base in 'ACGT'}  # Start with pseudo counts
        for base in s:
            base_counts[base] += 1
        total = sum(base_counts.values())
        return {'': {base: count / total for base, count in base_counts.items()}}
    
    # Handle kth order
    context_dict = {"".join(ctx): {nuc: 1 for nuc in 'ACGT'} for ctx in product('ACGT', repeat=k)}  # Pseudo counts
    for i in range(len(s) - k):
        context = s[i:i+k]
        following_nuc = s[i+k]
        context_dict[context][following_nuc] += 1

    # Convert counts to probabilities
    for context, counts in context_dict.items():
        total = sum(counts.values())
        context_dict[context] = {nuc: count / total for nuc, count in counts.items()}

    return context_dict

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    zeroth = context_pseudo_probabilities(s, 0)['']
    kth = context_pseudo_probabilities(s, k)
    print(f"zeroth: {zeroth}")
    for ctx, probs in kth.items():
        print(f"{ctx}: {probs}")

    mc = MarkovChain(zeroth, kth, k)
    print("\nGenerated sequence:", mc.generate(20))


zeroth: {'A': 0.32, 'C': 0.16, 'G': 0.24, 'T': 0.28}
AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666

### Idea of solution

Idea behind this solution is to build a probabilistic model that accurately reflects the nucleotide distribution and dependencies in a given sequence, using pseudo-counts to ensure that the model is robust to unseen contexts or nucleotides. This model can then be used to generate new sequences or to assess the likelihood of observed sequences, providing valuable insights into the underlying genetic patterns.

### Discussion

fill in 

14. Write class ```MarkovProb``` that given the $k$-th order Markov chain developed above to the constructor, its method ```probability``` computes the probability of a given input DNA sequence.

In [27]:
import numpy as np

class MarkovProb:
    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth
        
    def probability(self, s):
        if len(s) < self.k:
            return 0.0  # The sequence is too short to apply the k-th order model
        
        # Initialize the probability with the zeroth-order model for the first k nucleotides
        prob = np.product([self.zeroth[nuc] for nuc in s[:self.k]])
        
        # Multiply by the conditional probabilities for each subsequent nucleotide
        for i in range(len(s) - self.k):
            context = s[i:i+self.k]
            next_nuc = s[i+self.k]
            # Use kth order probabilities if context is found, otherwise treat as a rare event (very low probability)
            context_prob = self.kth.get(context, {next_nuc: 1 / (sum(self.zeroth.values()) * len(self.zeroth))})
            prob *= context_prob.get(next_nuc, 1 / (sum(self.zeroth.values()) * len(self.zeroth)))  # Handle rare event
        
        return prob

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    mc_prob = MarkovProb(k, zeroth, kth)
    test_seq = "ATGATATCATCGACGATGTAG"
    print(f"Probability of sequence {test_seq} is {mc_prob.probability(test_seq)}")


Probability of sequence ATGATATCATCGACGATGTAG is 2.831270190340017e-10


/tmp/ipykernel_22559/3421395266.py:33: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  print(f"Probability of sequence {test_seq} is {mc_prob.probability(test_seq)}")


### Idea of solution

The idea of this solution is to provide a quantifiable measure of how likely a sequence is, given a Markov model of nucleotide sequences. This can be particularly useful in bioinformatics for sequence analysis, alignment, modeling genetic sequences' behaviors, and understanding the statistical properties of genetic data.

### Discussion

fill in

With the last assignment you might end up in trouble with precision, as multiplying many small probabilities gives a really small number in the end. There is an easy fix by using so-called log-transform. 
Consider computation of $P=s_1 s_2 \cdots s_n$, where $0\leq s_i\leq 1$ for each $i$. Taking logarithm in base 2 from both sides gives $\log _2 P= \log _2 (s_1 s_2 \cdots s_n)=\log_2 s_1 + \log_2 s_2 + \cdots \log s_n= \sum_{i=1}^n \log s_i$, with repeated application of the property that the logarithm of a multiplication of two numbers is the sum of logarithms of the two numbers taken separately. The results is abbreviated as log-probability.

15. Write class ```MarkovLog``` that given the $k$-th order Markov chain developed above to the constructor, its method ```log_probability``` computes the log-probability of a given input DNA sequence. Run your program with $T=$ `ATGATATCATCGACGATGTAG` and $k=2$.

In [28]:
import numpy as np

class MarkovLog(object):

    def __init__(self, k, zeroth, kth):
        self.k = k
        self.zeroth = zeroth
        self.kth = kth

    def log_probability(self, s):
        if len(s) < self.k:
            return np.log(0)  # Log(0) for sequences shorter than k
        
        # Compute the log-probability for the first k nucleotides using the zeroth order model
        log_prob = np.sum([np.log(self.zeroth[nuc]) for nuc in s[:self.k]])
        
        # Add the log of conditional probabilities for each subsequent nucleotide
        for i in range(len(s) - self.k):
            context = s[i:i+self.k]
            next_nuc = s[i+self.k]
            context_prob = self.kth.get(context, {next_nuc: 1 / (sum(self.zeroth.values()) * len(self.zeroth))})
            log_prob += np.log(context_prob.get(next_nuc, 1 / (sum(self.zeroth.values()) * len(self.zeroth))))
        
        return log_prob

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    kth = context_pseudo_probabilities(s, k)
    zeroth = context_pseudo_probabilities(s, 0)[""]
    mc_log = MarkovLog(k, zeroth, kth)
    test_seq = "ATGATATCATCGACGATGTAG"
    print(f"Log probability of sequence {test_seq} is {mc_log.log_probability(test_seq)}")


Log probability of sequence ATGATATCATCGACGATGTAG is -21.985125488470754


### Idea of solution

MarkovLog solution is to compute the log-probability of observing a specific DNA sequence under a given k-th order Markov chain model. This approach offers several computational and numerical advantages over directly calculating the sequence's probability, especially for long sequences or when dealing with very small probabilities.

### Discussion

fill in

Finally, if you try to use the code so far for very large inputs, you might observe that the concatenation of symbols following a context occupy considerable amount of space. This is unnecessary, as we only need the frequencies. 

16. Optimize the space requirement of your code from exercise 13 for the $k$-th order Markov chain by replacing the concatenations by direct computations of the frequencies. Implement this as the
  ```better_context_probabilities``` function.

In [29]:
from itertools import product

def better_context_probabilities(s, k):
    # Initialize context dictionary with pseudo counts to handle unseen k-mers
    context_dict = {
        ''.join(ctx): {nuc: 1 for nuc in 'ACGT'} for ctx in product('ACGT', repeat=k)
    }

    # Count occurrences directly without concatenation
    for i in range(len(s) - k):
        context = s[i:i+k]
        next_nuc = s[i+k]
        if context in context_dict:
            context_dict[context][next_nuc] += 1
        else:
            # Initialize with pseudo counts if the context is somehow not in the dictionary (shouldn't happen with pseudo counts initialization)
            context_dict[context] = {nuc: 1 for nuc in 'ACGT'}
            context_dict[context][next_nuc] += 1

    # Convert counts to probabilities
    for context, counts in context_dict.items():
        total = sum(counts.values())
        context_dict[context] = {nuc: count / total for nuc, count in counts.items()}

    return context_dict

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    d = better_context_probabilities(s, k)
    for context, probabilities in d.items():
        print(f"{context}: {probabilities}")


AA: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AC: {'A': 0.2, 'C': 0.2, 'G': 0.4, 'T': 0.2}
AG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
AT: {'A': 0.2222222222222222, 'C': 0.3333333333333333, 'G': 0.3333333333333333, 'T': 0.1111111111111111}
CA: {'A': 0.2, 'C': 0.2, 'G': 0.2, 'T': 0.4}
CC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
CG: {'A': 0.5, 'C': 0.16666666666666666, 'G': 0.16666666666666666, 'T': 0.16666666666666666}
CT: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GA: {'A': 0.14285714285714285, 'C': 0.2857142857142857, 'G': 0.14285714285714285, 'T': 0.42857142857142855}
GC: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GG: {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}
GT: {'A': 0.4, 'C': 0.2, 'G': 0.2, 'T': 0.2}
TA: {'A': 0.16666666666666666, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.3333333333333333}
TC: {'A': 0.3333333333333333, 'C': 0.16666666666666666, 'G': 0.3333333333333333, 'T': 0.16666666666666666}
TG: {'A': 0.3333333333333333, 'C': 0.16666666666666

### Idea of solution

Idea behind the better_context_probabilities function is to efficiently calculate the probabilities of each nucleotide following a particular sequence (context) of k nucleotides in a given DNA sequence. This function is designed to be space-efficient, avoiding the need to store large amounts of intermediate data.

### Discussion

fill in

While the earlier approach of explicit concatenation of symbols following a context suffered from inefficient use of space, it does have a benefit of giving another much simpler strategy to sample from the distribution: 
observe that an element of the concatenation taken uniformly randomly is sampled exactly with the correct probability. 

17. Revisit the solution 12 and modify it to directly sample from the concatenation of symbols following a context. The function ```np.random.choice``` may be convenient here. Implement the modified version as the new `SimpleMarkovChain` class.

In [30]:
import numpy as np
import random
from collections import defaultdict

class SimpleMarkovChain(object):
    def __init__(self, s, k):
        self.k = k
        self.contexts = defaultdict(list)
        # Build the context list
        for i in range(len(s) - k):
            self.contexts[s[i:i+k]].append(s[i+k])

    def generate(self, n, seed=None):
        if seed is not None:
            np.random.seed(seed)
            random.seed(seed)

        # Start by choosing a random k-mer as the initial context
        initial_context = random.choice(list(self.contexts.keys()))
        sequence = initial_context

        for _ in range(n - self.k):
            context = sequence[-self.k:]
            # If the context is not found (which shouldn't happen with our training data), choose a random context
            if context not in self.contexts or not self.contexts[context]:
                context = random.choice(list(self.contexts.keys()))

            # Sample the next nucleotide from the list of observed nucleotides following the current context
            next_nucleotide = np.random.choice(self.contexts[context])
            sequence += next_nucleotide

        return sequence

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    n = 10
    seed = 7
    mc = SimpleMarkovChain(s, k)
    print(mc.generate(n, seed))


CATGTATCGA


### Idea of solution

The SimpleMarkovChain class is designed to mimic how DNA sequences naturally evolve, using patterns found in a given sample sequence.

### Discussion

fill in

## $k$-mer index

Our $k$-th order Markov chain can now be modified to a handy index structure called $k$-mer index. This index structure associates to each $k$-mer its list of occurrence positions in DNA sequence $T$.  Given a query $k$-mer $W$, one can thus easily list all positions $i$ with  $T[i..k-1]=W$.

18. Implement function ```kmer_index``` inspired by your earlier code for the $k$-th order Markov chain. Test your program with `ATGATATCATCGACGATGTAG` and $k=2$.

In [31]:
def kmer_index(s, k):
    index = {}
    # Iterate over the sequence to extract k-mers and their positions
    for i in range(len(s) - k + 1):
        kmer = s[i:i+k]
        if kmer in index:
            index[kmer].append(i)
        else:
            index[kmer] = [i]
    return index

if __name__ == '__main__':
    k = 2
    s = "ATGATATCATCGACGATGTAG"
    print("Using string:")
    print(s)
    print("".join([str(i % 10) for i in range(len(s))]))
    
    print(f"\n{k}-mer index is:")
    d = kmer_index(s, k)
    for kmer, positions in d.items():
        print(f"{kmer}: {positions}")


Using string:
ATGATATCATCGACGATGTAG
012345678901234567890

2-mer index is:
AT: [0, 3, 5, 8, 15]
TG: [1, 16]
GA: [2, 11, 14]
TA: [4, 18]
TC: [6, 9]
CA: [7]
CG: [10, 13]
AC: [12]
GT: [17]
AG: [19]


### Idea of solution

The idea behind the kmer_index solution is to create a quick-reference guide or "index" for each short sequence (k-mer) within a longer DNA sequence. This index helps you rapidly find where each k-mer occurs in the DNA sequence, much like an index in a book that helps you find information quickly without reading the entire book.

### Discussion

fill in

## Comparison of probability distributions

Now that we know how to learn probability distributions from data, we might want to compare two such distributions, for example, to test if our programs work as intended. 

Let $P=\{p_1,p_2,\ldots, p_n\}$ and $Q=\{q_1,q_2,\ldots, q_n\}$ be two probability distributions for the same set of $n$ events. This means $\sum_{i=1}^n p_i=\sum_{i=1}^n q_i=1$, $0\leq p_j \leq 1$, and $0\leq q_j \leq 1$ for each event $j$. 

*Kullback-Leibler divergence* is a measure $d()$ for the *relative entropy* of $P$ with respect to $Q$ defined as 
$d(P||Q)=\sum_{i=1}^n p_i \log\frac{p_i}{q_i}$.


This measure is always non-negative, and 0 only when $P=Q$. It can be interpreted as the gain of knowing $Q$ to encode $P$. Note that this measure is not symmetric.

19. Write function ```kullback_leibler``` to compute $d(P||Q)$. Test your solution by generating a random RNA sequence
  encoding the input protein sequence according to the input codon adaptation probabilities.
  Then you should learn the codon adaptation probabilities from the RNA sequence you generated.
  Then try the same with uniformly random RNA sequences (which don't have to encode any
  specific protein sequence). Compute the relative entropies between the
  three distribution (original, predicted, uniform) and you should observe a clear difference.
  Because $d(P||Q)$ is not symmetric, you can either print both $d(P||Q)$ and $d(Q||P)$,
  or their average.
  
  This problem may be fairly tricky. Only the `kullback_leibler` function is automatically tested. The codon probabilities is probably a useful helper function. The main guarded section can be completed by filling out the `pass` sections using tooling from previous parts and fixing the *placeholder* lines.

In [32]:
def codon_probabilities(rna):
    """
    Given an RNA sequence, simply calculates the proability of
    all 3-mers empirically based on the sequence
    """
    return {"".join(codon): 0 for codon in product("ACGU", repeat=3)}
    
def kullback_leibler(p, q):
    """
    Computes Kullback-Leibler divergence between two distributions.
    Both p and q must be dictionaries from events to probabilities.
    The divergence is defined only when q[event] == 0 implies p[event] == 0.
    """
    return np.nan

if __name__ == '__main__':
    aas = list("*ACDEFGHIKLMNPQRSTVWY") # List of amino acids
    n = 10000
    
    # generate a random protein and some associated rna
    protein = "".join(choice(aas, n))    
    pass
    
    # Maybe check that converting back to protein results in the same sequence
    pass
    
    # Calculate codon probabilities of the rna sequence
    cp_predicted = codon_probabilities("<rna sequence>") # placeholder call
    
    # Calculate codon probabilities based on the codon usage table
    cp_orig = {"".join(codon): 0 for codon in product("ACGU", repeat=3)} # placeholder dict
    
    # Create a completely random RNA sequence and get the codon probabilities
    pass
    cp_uniform = codon_probabilities("<random rna sequence>") # placeholder call
    
    print("d(original || predicted) =", kullback_leibler(cp_orig, cp_predicted))
    print("d(predicted || original) =", kullback_leibler(cp_predicted, cp_orig))
    print()
    print("d(original || uniform) =", kullback_leibler(cp_orig, cp_uniform))
    print("d(uniform || original) =", kullback_leibler(cp_uniform, cp_orig))
    print()
    print("d(predicted || uniform) =", kullback_leibler(cp_predicted, cp_uniform))
    print("d(uniform || predicted) =", kullback_leibler(cp_uniform, cp_predicted))

SyntaxError: incomplete input (784837457.py, line 40)

### Idea of solution

fill in

### Discussion

fill in

## Stationary and equilibrium distributions (extra)

Let us consider a Markov chain of order one on the set of nucleotides.
Its transition probabilities can be expressed as a $4 \times 4$ matrix
$P=(p_{ij})$, where the element $p_{ij}$ gives the probability of the $j$th nucleotide
on the condition the previous nucleotide was the $i$th. An example of a transition matrix
is

\begin{array}{l|rrrr}
 &     A &    C &     G &    T \\
\hline
A &  0.30 &  0.0 &  0.70 &  0.0 \\
C &  0.00 &  0.4 &  0.00 &  0.6 \\
G &  0.35 &  0.0 &  0.65 &  0.0 \\
T &  0.00 &  0.2 &  0.00 &  0.8 \\
\end{array}.

A distribution $\pi=(\pi_1,\pi_2,\pi_3,\pi_4)$ is called *stationary*, if
$\pi = \pi P$ (the product here is matrix product).

20. Write function ```get_stationary_distributions``` that gets a transition matrix as parameter,
  and returns the list of stationary distributions. You can do this with NumPy by
  first taking transposition of both sides of the above equation to get equation
  $\pi^T = P^T \pi^T$. Using numpy.linalg.eig take all eigenvectors related to
  eigenvalue 1.0. By normalizing these vectors to sum up to one get the stationary distributions
  of the original transition matrix. In the ```main``` function print the stationary distributions
  of the above transition matrix.

In [40]:
import numpy as np

def get_stationary_distributions(transition):
    """
    The function gets a transition matrix of a degree one Markov chain as parameter.
    It returns a list of stationary distributions, in vector form, for that chain.
    """
    # Find eigenvalues and eigenvectors of the transposed transition matrix
    eigenvalues, eigenvectors = np.linalg.eig(transition.T)
    
    # Filter eigenvectors corresponding to eigenvalue 1 (or very close to it due to floating-point precision)
    stationary = [vec / np.sum(vec) for vec, val in zip(eigenvectors.T, eigenvalues) if np.isclose(val, 1.0)]
    
    # Ensure that each vector sums to one (normalization)
    return [vec.real for vec in stationary]  # Convert to real in case of complex numbers due to computation
    
if __name__ == "__main__":
    transition = np.array([[0.3, 0, 0.7, 0],
                           [0, 0.4, 0, 0.6],
                           [0.35, 0, 0.65, 0],
                           [0, 0.2, 0, 0.8]])
    stationary_distributions = get_stationary_distributions(transition)
    print("\n".join(", ".join(f"{pv:+.3f}" for pv in p) for p in stationary_distributions))


+0.333, +0.000, +0.667, +0.000
-0.000, +0.250, -0.000, +0.750


### Idea of solution


### Discussion


21. Implement the `kl_divergence` function below so that the main guarded code runs properly. Using your modified Markov chain generator generate a nucleotide sequence $s$ of length $10\;000$. Choose prefixes of $s$ of lengths $1, 10, 100, 1000$, and $10\;000$. For each of these prefixes find out their nucleotide distribution (of order 0) using your earlier tool. Use 1 as the pseudo count. Then, for each prefix, compute the KL divergence between the initial distribution and the normalized nucleotide distribution.

In [41]:
import numpy as np

def kl_divergence(p, q):
    """
    Calculate the KL divergence between two distributions.
    Ensure that p and q are numpy arrays.
    """
    return np.sum(p * np.log(p / q), where=(p != 0))

def empirical_distribution(sequence, pseudo_count=1):
    """
    Calculate the empirical distribution of nucleotides in the sequence.
    Add pseudo_count to each nucleotide count for smoothing.
    """
    counts = np.array([sequence.count(nuc) for nuc in 'ACGT']) + pseudo_count
    return counts / counts.sum()

def generate_sequence(initial, transition, length):
    """
    Generate a sequence of given length using the initial distribution and transition matrix.
    """
    sequence = ''
    current_state = np.random.choice(['A', 'C', 'G', 'T'], p=initial)
    sequence += current_state

    for _ in range(1, length):
        next_state = np.random.choice(['A', 'C', 'G', 'T'], p=transition['ACGT'.index(current_state)])
        sequence += next_state
        current_state = next_state

    return sequence

def kl_divergences(initial, transition):
    lengths = [1, 10, 100, 1000, 10000]
    divergences = []

    for length in lengths:
        sequence = generate_sequence(initial, transition, length)
        empirical_dist = empirical_distribution(sequence)
        divergence = kl_divergence(initial, empirical_dist)
        divergences.append((length, divergence))

    return divergences

# Main execution
if __name__ == "__main__":
    transition = np.array([[0.3, 0, 0.7, 0],
                           [0, 0.4, 0, 0.6],
                           [0.35, 0, 0.65, 0],
                           [0, 0.2, 0, 0.8]])
    print("Transition probabilities are:")
    print(transition)
    stationary_distributions = get_stationary_distributions(transition)
    print("Stationary distributions:")
    print(np.stack(stationary_distributions))
    initial = stationary_distributions[0]  # Assuming the first one is what we want to use
    print("Using [{}] as initial distribution\n".format(", ".join(f"{v:.2f}" for v in initial)))
    results = kl_divergences(initial, transition)
    for prefix_length, divergence in results:
        print("KL divergence of stationary distribution prefix " \
              "of length {:5d} is {:.8f}".format(prefix_length, divergence))


Transition probabilities are:
[[0.3  0.   0.7  0.  ]
 [0.   0.4  0.   0.6 ]
 [0.35 0.   0.65 0.  ]
 [0.   0.2  0.   0.8 ]]
Stationary distributions:
[[ 0.33333333  0.          0.66666667  0.        ]
 [-0.          0.25       -0.          0.75      ]]
Using [0.33, 0.00, 0.67, 0.00] as initial distribution



/tmp/ipykernel_22559/832333814.py:8: RuntimeWarning: divide by zero encountered in log
  return np.sum(p * np.log(p / q), where=(p != 0))
/tmp/ipykernel_22559/832333814.py:8: RuntimeWarning: invalid value encountered in multiply
  return np.sum(p * np.log(p / q), where=(p != 0))


KL divergence of stationary distribution prefix of length     1 is 0.51082562
KL divergence of stationary distribution prefix of length    10 is 0.21078369
KL divergence of stationary distribution prefix of length   100 is 0.02303352
KL divergence of stationary distribution prefix of length  1000 is 0.00217722
KL divergence of stationary distribution prefix of length 10000 is 0.00020645


### Idea of solution

fill in

### Discussion
fill in

22. Implement the following in the ```main``` function.
Find the stationary distribution for the following transition matrix:  

\begin{array}{ l | r r r r}
 & A &     C &     G &     T \\
\hline
A &  0.30 &  0.10 &  0.50 &  0.10 \\
C &  0.20 &  0.30 &  0.15 &  0.35 \\
G &  0.25 &  0.15 &  0.20 &  0.40 \\
T &  0.35 &  0.20 &  0.40 &  0.05 \\
\end{array}

Since there is only one stationary distribution, it is called the *equilibrium distribution*.
Choose randomly two nucleotide distributions. You can take these from your sleeve or
sample them from the Dirichlet distribution. Then for each of these distributions
as the initial distribution of the Markov chain, repeat the above experiment.

The `main` function should return tuples, where the first element is the (random) initial distribution and the second element contains the results as a list of tuples where the first element is the kl divergence and the second element the empirical nucleotide distribution, for the different prefix lengths.

The state distribution should converge to the equilibrium distribution no matter how we
start the Markov chain! That is the last line of the tables should have KL-divergence very close to $0$ and an empirical distribution very close to the equilibrium distribution.


In [42]:
import numpy as np

def main(transition, equilibrium_distribution):
    # Generate two random initial distributions from the Dirichlet distribution
    initial_distributions = np.random.dirichlet(alpha=[1, 1, 1, 1], size=2)
    
    results = []
    for initial_distribution in initial_distributions:
        # For each initial distribution, generate sequences and compute KL divergences and empirical distributions
        kl_divergences_and_distributions = []
        for length in [1, 10, 100, 1000, 10000]:
            sequence = generate_sequence(initial_distribution, transition, length)
            empirical_dist = empirical_distribution(sequence)
            kl_divergence_value = kl_divergence(empirical_dist, equilibrium_distribution)
            kl_divergences_and_distributions.append((kl_divergence_value, empirical_dist))
        results.append((initial_distribution, kl_divergences_and_distributions))
    
    return results

if __name__ == "__main__":
    transition = np.array([[0.3, 0.1, 0.5, 0.1],
                           [0.2, 0.3, 0.15, 0.35],
                           [0.25, 0.15, 0.2, 0.4],
                           [0.35, 0.2, 0.4, 0.05]])
    print("Transition probabilities are:", transition, sep="\n")

    stationary_distributions = get_stationary_distributions(transition)
    equilibrium_distribution = stationary_distributions[0]
    print("Equilibrium distribution:")
    print(equilibrium_distribution)

    experiment_results = main(transition, equilibrium_distribution)
    for initial_distribution, results in experiment_results:
        print("\nUsing {} as initial distribution:".format(initial_distribution))
        print("kl-divergence   empirical distribution")
        for kl_divergence_value, empirical_dist in results:
            print("{:.11f}   {}".format(kl_divergence_value, empirical_dist))


Transition probabilities are:
[[0.3  0.1  0.5  0.1 ]
 [0.2  0.3  0.15 0.35]
 [0.25 0.15 0.2  0.4 ]
 [0.35 0.2  0.4  0.05]]
Equilibrium distribution:
[0.27803345 0.17353238 0.32035021 0.22808396]

Using [0.2306959  0.20041794 0.351002   0.21788416] as initial distribution:
kl-divergence   empirical distribution
0.09298662570   [0.2 0.2 0.2 0.4]
0.01959911268   [0.21428571 0.14285714 0.35714286 0.28571429]
0.03289168640   [0.17307692 0.20192308 0.33653846 0.28846154]
0.00364981923   [0.29282869 0.14243028 0.33366534 0.2310757 ]
0.00001820215   [0.27568972 0.17483007 0.32177129 0.22770892]

Using [0.03354826 0.33311738 0.300237   0.33309736] as initial distribution:
kl-divergence   empirical distribution
0.09298662570   [0.2 0.2 0.2 0.4]
0.03509184433   [0.35714286 0.21428571 0.21428571 0.21428571]
0.01261005760   [0.32692308 0.125      0.33653846 0.21153846]
0.00020162134   [0.28486056 0.16733068 0.3187251  0.22908367]
0.00013437712   [0.27638944 0.16823271 0.32337065 0.2320072 ]


### Idea of solution

The idea behind this solution is to demonstrate and verify a fundamental property of Markov chains: regardless of the initial distribution, the state distribution of a Markov chain will converge to its equilibrium (or stationary) distribution as the chain progresses over many steps.

### Discussion
fill in